In [ ]:
USERNAME = 
PASSWORD = 

In [1]:
import os
import time
from datetime import date, timedelta
from os import path

import Driver
import pandas as pd
from bs4 import BeautifulSoup

### waitRand
Waits a random amount of seconds then prints time waited.

Default time is 1-3 seconds

In [3]:
from numpy.random import randint

def waitRand(maxTime=3):
    waitTime = randint(1,maxTime+1)
    time.sleep(waitTime)
    if waitTime > 1:
        print("Waited for {} seconds.".format(waitTime))
    else:
        print("Waited for {} second.".format(waitTime))

### clickAndLog
Clicks on a link, then returns the source.

In [4]:
def clickAndLog(linkText, method = 'link text'):
    waitRand()
    driver.find_element(method, linkText).click()
    print("Got {}.".format(linkText))
    return driver.page_source

In [5]:
driver = Driver.Driver()
driver.login(USERNAME, PASSWORD)

## Scrape Pages (Pure HTML)

In [ ]:
production = clickAndLog("Trends")

buildings = clickAndLog("Number of buildings")

pricesCalc = clickAndLog("Calculation of market prices")

marketSales = clickAndLog("Sales on the market")

shopSales = clickAndLog("Sales in shops")

#there are two links named Offices, so we must locate by xpath
offices = clickAndLog(r"/html/body/div[2]/div[2]/table/tbody/tr/td[2]/table/tbody/tr[2]/td/div[1]/nobr[7]/a", method = "xpath")
                          
prices = clickAndLog("Prices")

volume = clickAndLog("Buy goods")

Waited for 1 second.
Got Trends.
Waited for 1 second.
Got Number of buildings.
Waited for 3 seconds.
Got Calculation of market prices.
Waited for 1 second.
Got Sales on the market.
Waited for 1 second.
Got Sales in shops.
Waited for 2 seconds.
Got /html/body/div[2]/div[2]/table/tbody/tr/td[2]/table/tbody/tr[2]/td/div[1]/nobr[7]/a.
Waited for 3 seconds.


In [ ]:
cookies = driver.get_cookies()
print('Done.')
driver.close()

yesterday = date.today() - timedelta(1)
today = date.today()
print('Recorded date of data collection.')

## Convert HTML to dataframes

In [ ]:
df = pd.read_html(production, match="Goods", header=4, index_col=0)
production_df = df[0].dropna()

In [ ]:
df = pd.read_html(buildings, match="Factory", header=4, index_col=1)
buildings_df = df[0].drop(columns=['Unnamed: 0']).dropna()
buildings_df = buildings_df.drop(['In total','Shop','Offices']) #drop totals, shop, office

In [ ]:
df = pd.read_html(pricesCalc, match="Goods", header=4, index_col=0)
pricesCalc_df = df[0].dropna()

In [ ]:
df = pd.read_html(marketSales, match="Goods", header=4, index_col=0)
marketSales_df = df[0].dropna()

In [ ]:
df = pd.read_html(shopSales, match="Goods", header=4, index_col=0)
shopSales_df = df[0].dropna()

In [ ]:
df = pd.read_html(offices, match="Office jobs hours created nationwide", header=4, index_col=0)
offices_df = df[0].dropna()

In [ ]:
pricesTable = BeautifulSoup(prices).find_all("table")[2]
df = pd.read_html(str(pricesTable))[0]
df = df.drop([0,1,2,3,4])
df = df.set_index(0)
df = df.rename(columns=df.iloc[0])
df = df.drop(['Goods'])
df.index.names = ['Goods']
df = df.iloc[:,-1]
prices_df = df.dropna()
prices_df.name = pd.to_datetime(prices_df.name)

In [ ]:
df = pd.read_html(volume)

## Append existing CSV files

In [ ]:
def convert(input_df):
    """
    Transposes data and converts to int
    
    Renames index to actual dates
    
    Drops today data because it is incomplete
    """
    dfT = input_df.T.astype(int)
    
    dfT.rename(index={'Yesterday':yesterday},inplace=True)
    dfT.rename(index={'Today':today},inplace=True)
    dfT.index = pd.to_datetime(dfT.index)
    dfT = dfT.drop([today])
    return dfT

In [ ]:
def save_data(input_df, csv_path):
    """
    Creates new csv or appends data to existing csv
    """
    if not path.exists(csv_path):
        input_df.to_csv(csv_path)
        print('File did not exist, created new file and saved data.')
    else:
        file = pd.read_csv(csv_path, index_col=0)
        file.index = pd.to_datetime(file.index)
        file = file.append(input_df)
        file = file.drop_duplicates().sort_index()
        file.to_csv(csv_path)
        print('Data successfully appended to existing file.')

In [ ]:
production_path = 'data/production.csv'

production_dfT = convert(production_df)
save_data(production_dfT, production_path)

In [ ]:
marketSales_path = 'data/marketSales.csv'

marketSales_dfT = convert(marketSales_df)
save_data(marketSales_dfT, marketSales_path)

In [ ]:
shopSales_path = 'data/shopSales.csv'

shopSales_dfT = convert(shopSales_df)
save_data(shopSales_dfT, shopSales_path)

In [ ]:
date_path = 'data/' + 'buildingData/' + str(today)
os.mkdir(date_path)
pricesCalc_df.to_csv('{}/pricesCalc.csv'.format(date_path))
offices_df.to_csv('{}/offices.csv'.format(date_path))
buildings_df.T.to_csv('{}/offices.csv'.format(date_path))

In [ ]:
prices_file = os.path.join("data", "prices.csv")
if not os.path.exists(prices_file):
    with open(prices_file, "w") as f:
        f.write(",Alcohol,Bread,Clothes,Cotton,Flour,Fresh fish,Frozen fish,Furniture,Gasoline,Marble,Newspapers,Oil,Paper,Pigs,Planks,Plastic,Sausages,Stone,Thread,Toys,Wheat,Wood")
df = pd.read_csv(prices_file, index_col=0)
df.index = pd.to_datetime(df.index)
if prices_df.name not in df.index:
    df = df.append(prices_df)
    df.to_csv(prices_file)
    print('Successfully Appended')
else:
    print('Today\'s info already in spreadsheet')
